In [1]:
                        ##-- MODEL CONSTRUCTION --##
# Importing libraries

import pandas as pd
import getpass
import math
import numpy as np
#from tqdm import trange
import gc
#import matplotlib.pyplot as plt
#import seaborn as sns
#from sklearn.ensemble import IsolationForest

if getpass.getuser() == "asus":
    file_path = "C:/Users/asus/Desktop/trendyol_data_v2/"
    
if getpass.getuser() == "your computers name":
    file_path = "C:/Users/..desiredlocation..."

In [2]:
main_data = pd.read_csv(file_path + 'main_data_v5.csv',index_col=0)

C:\Users\asus\AppData\Local\R-MINI~1\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
main_data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10772622 entries, 0 to 10772621
Data columns (total 50 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   user_id                   int64  
 1   is_elite_user             int64  
 2   supplier_id               int64  
 3   order_parent_id           int64  
 4   product_content_id        int64  
 5   product_variant_id        int64  
 6   discounted_price          float64
 7   ship_cost                 float64
 8   is_wallet_trx             int64  
 9   is_saved_card_trx         int64  
 10  is_returned               float64
 11  coupon_used               int64  
 12  is_different_sizes        int64  
 13  is_discounted             int64  
 14  promotion_percent         float64
 15  is_shipcost_incurred      int64  
 16  shipping_percent          float64
 17  Erkek                     int64  
 18  GenderlessItem            int64  
 19  Kadın                     int64  
 20  Kadın_Kız             

In [4]:
main_data.head(3)

,user_id,is_elite_user,supplier_id,order_parent_id,product_content_id,product_variant_id,discounted_price,ship_cost,is_wallet_trx,is_saved_card_trx,...,Turkuaz,Turuncu,Yeşil,return_prob_from_answers,return_prob_from_reviews,suppliers_return_rate,dispute_label,returnRate,return_label,gender
0,3941815,0,4662,595758968,95503167,159147197,51.94,8.88,0,1,...,0,0,0,0.0,0.000000,0.003045,0.0,0.098178,0,Bayan
1,3941815,0,163,595758968,36313798,64622938,59.97,9.40,0,1,...,0,0,1,0.0,0.047619,0.000545,0.0,0.050737,0,Bayan
2,3941815,0,157306,595758968,36888320,65612791,49.99,10.67,0,1,...,0,0,0,0.0,0.000000,0.000744,1.0,0.093355,0,Bayan


In [5]:
dummy_gender = pd.get_dummies(main_data['gender'])                          
main_data = pd.merge(
    left=main_data,
    right=dummy_gender,
    left_index=True,
    right_index=True,
)                           
main_data = main_data.drop(['gender'],axis=1) 

In [9]:
main_data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10772622 entries, 0 to 10772621
Data columns (total 51 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   user_id                   int64  
 1   is_elite_user             int64  
 2   supplier_id               int64  
 3   order_parent_id           int64  
 4   product_content_id        int64  
 5   product_variant_id        int64  
 6   discounted_price          float64
 7   ship_cost                 float64
 8   is_wallet_trx             int64  
 9   is_saved_card_trx         int64  
 10  is_returned               float64
 11  coupon_used               int64  
 12  is_different_sizes        int64  
 13  is_discounted             int64  
 14  promotion_percent         float64
 15  is_shipcost_incurred      int64  
 16  shipping_percent          float64
 17  Erkek                     int64  
 18  GenderlessItem            int64  
 19  Kadın                     int64  
 20  Kadın_Kız             

Currently the memory usage is 4.3 gb, although the file size is 2.8 gb. \
Considering my RAM, 4.6 gb memory usage is OK but I may have to drop some columns \
(possibly one-hot encoded ones) from my dataframe. Will see in a minute.

In [8]:
main_data.head(1)

,user_id,is_elite_user,supplier_id,order_parent_id,product_content_id,product_variant_id,discounted_price,ship_cost,is_wallet_trx,is_saved_card_trx,...,Turuncu,Yeşil,return_prob_from_answers,return_prob_from_reviews,suppliers_return_rate,dispute_label,returnRate,return_label,Bay,Bayan
0,3941815,0,4662,595758968,95503167,159147197,51.94,8.88,0,1,...,0,0,0.0,0.0,0.003045,0.0,0.098178,0,0,1


IDEA 2:
user id  --> user demographics
product content id  --> product features
order parent id --> is_different_sizes column (will be derived within the test data)
un-concatenate the predictor column in test data and join these information stated above, train your model with training data, 
then fit the model to test data to do the prediction, then 
delete all columns in test data except user_id, product_content_id, order_parent_id and is_returned (prediction).
concatenate three id's. then you are done.\
But still I couldn't figure out how to exploit supplier_id.

We can bring all information associated with user_id, product_content_id \
and order_id (only 'is_different_sizes' column), but how can I bring those 4 return-possibility-related \
columns associated with respective suppliers (suppliers_return_rate, dispute_label, returnRate, return_label)?

Maybe we can combine these 4 columns into a single one, and relate this new feature to product_content_id.

In [20]:
main_data['return_probability_supplier_1'] = main_data.groupby('product_content_id')['suppliers_return_rate'].transform('mean')

In [21]:
main_data.head(3)

,user_id,is_elite_user,supplier_id,order_parent_id,product_content_id,product_variant_id,discounted_price,ship_cost,is_wallet_trx,is_saved_card_trx,...,Yeşil,return_prob_from_answers,return_prob_from_reviews,suppliers_return_rate,dispute_label,returnRate,return_label,Bay,Bayan,return_probability_supplier_1
0,3941815,0,4662,595758968,95503167,159147197,51.94,8.88,0,1,...,0,0.0,0.000000,0.003045,0.0,0.098178,0,0,1,0.003045
1,3941815,0,163,595758968,36313798,64622938,59.97,9.40,0,1,...,1,0.0,0.047619,0.000545,0.0,0.050737,0,0,1,0.000545
2,3941815,0,157306,595758968,36888320,65612791,49.99,10.67,0,1,...,0,0.0,0.000000,0.000744,1.0,0.093355,0,0,1,0.000744


In [22]:
main_data['return_probability_supplier_2'] = main_data.groupby('product_content_id')['dispute_label'].transform('mean')

In [23]:
main_data['return_probability_supplier_3'] = main_data.groupby('product_content_id')['returnRate'].transform('mean')

In [24]:
main_data['return_probability_supplier_4'] = main_data.groupby('product_content_id')['return_label'].transform('mean')

In [27]:
main_data['overall_return_prob_product'] = main_data[['return_probability_supplier_4', 
'return_probability_supplier_3','return_probability_supplier_2','return_probability_supplier_1']].mean(axis=1)

In [28]:
main_data.head(1)

,user_id,is_elite_user,supplier_id,order_parent_id,product_content_id,product_variant_id,discounted_price,ship_cost,is_wallet_trx,is_saved_card_trx,...,dispute_label,returnRate,return_label,Bay,Bayan,return_probability_supplier_1,return_probability_supplier_2,return_probability_supplier_3,return_probability_supplier_4,overall_return_prob_product
0,3941815,0,4662,595758968,95503167,159147197,51.94,8.88,0,1,...,0.0,0.098178,0,0,1,0.003045,0.0,0.098178,0.0,0.025306


In [29]:
main_data_v2 = main_data.drop(['suppliers_return_rate','dispute_label','returnRate','return_label',
'return_probability_supplier_1','return_probability_supplier_2','return_probability_supplier_3',
'return_probability_supplier_4'],axis=1)

In [30]:
main_data_v2.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10772622 entries, 0 to 10772621
Data columns (total 48 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   user_id                      int64  
 1   is_elite_user                int64  
 2   supplier_id                  int64  
 3   order_parent_id              int64  
 4   product_content_id           int64  
 5   product_variant_id           int64  
 6   discounted_price             float64
 7   ship_cost                    float64
 8   is_wallet_trx                int64  
 9   is_saved_card_trx            int64  
 10  is_returned                  float64
 11  coupon_used                  int64  
 12  is_different_sizes           int64  
 13  is_discounted                int64  
 14  promotion_percent            float64
 15  is_shipcost_incurred         int64  
 16  shipping_percent             float64
 17  Erkek                        int64  
 18  GenderlessItem               int64  
 19

In [33]:
pd.set_option("display.max_columns", None)
main_data_v2.head(3)

,user_id,is_elite_user,supplier_id,order_parent_id,product_content_id,product_variant_id,discounted_price,ship_cost,is_wallet_trx,is_saved_card_trx,is_returned,coupon_used,is_different_sizes,is_discounted,promotion_percent,is_shipcost_incurred,shipping_percent,Erkek,GenderlessItem,Kadın,Kadın_Kız,Unisex,Altın,Bej,Beyaz,Bordo,Ekru,Gri,Gümüş,Haki,Kahverengi,Kırmızı,Lacivert,Mavi,Metalik,Mor,Multicolor,Pembe,Sarı,Siyah,Turkuaz,Turuncu,Yeşil,return_prob_from_answers,return_prob_from_reviews,Bay,Bayan,overall_return_prob_product
0,3941815,0,4662,595758968,95503167,159147197,51.94,8.88,0,1,0.0,0,0,1,0.349937,1,0.170966,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.000000,0,1,0.025306
1,3941815,0,163,595758968,36313798,64622938,59.97,9.40,0,1,0.0,0,0,1,0.538515,1,0.156745,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.047619,0,1,0.012821
2,3941815,0,157306,595758968,36888320,65612791,49.99,10.67,0,1,0.0,0,0,1,0.166694,1,0.213443,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.000000,0,1,0.273525


In [32]:
main_data_v2.isna().sum()

user_id                        0
is_elite_user                  0
supplier_id                    0
order_parent_id                0
product_content_id             0
product_variant_id             0
discounted_price               0
ship_cost                      0
is_wallet_trx                  0
is_saved_card_trx              0
is_returned                    0
coupon_used                    0
is_different_sizes             0
is_discounted                  0
promotion_percent              0
is_shipcost_incurred           0
shipping_percent               0
Erkek                          0
GenderlessItem                 0
Kadın                          0
Kadın_Kız                      0
Unisex                         0
Altın                          0
Bej                            0
Beyaz                          0
Bordo                          0
Ekru                           0
Gri                            0
Gümüş                          0
Haki                           0
Kahverengi

In [34]:
main_data_v2.describe()

,user_id,is_elite_user,supplier_id,order_parent_id,product_content_id,product_variant_id,discounted_price,ship_cost,is_wallet_trx,is_saved_card_trx,is_returned,coupon_used,is_different_sizes,is_discounted,promotion_percent,is_shipcost_incurred,shipping_percent,Erkek,GenderlessItem,Kadın,Kadın_Kız,Unisex,Altın,Bej,Beyaz,Bordo,Ekru,Gri,Gümüş,Haki,Kahverengi,Kırmızı,Lacivert,Mavi,Metalik,Mor,Multicolor,Pembe,Sarı,Siyah,Turkuaz,Turuncu,Yeşil,return_prob_from_answers,return_prob_from_reviews,Bay,Bayan,overall_return_prob_product
count,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07,1.077262e+07
mean,2.162149e+07,3.165927e-01,6.510637e+04,6.563724e+08,7.064945e+07,1.207076e+08,6.214491e+01,5.763882e+00,4.092687e-01,3.639821e-01,1.272053e-01,6.720175e-02,5.172343e-02,7.892287e-01,2.696161e-01,9.967416e-01,1.140409e-01,2.371838e-01,6.282593e-04,3.015979e-04,7.344331e-01,2.745330e-02,5.933560e-04,4.594926e-02,1.481329e-01,1.277368e-02,3.634965e-02,7.859971e-02,1.921538e-03,1.843785e-02,3.139588e-02,2.992976e-02,6.767925e-02,9.070364e-02,8.429703e-04,3.691553e-02,4.558779e-04,5.504008e-02,2.219153e-02,2.526788e-01,5.724604e-03,2.194081e-02,4.174332e-02,7.376493e-05,6.877569e-02,1.378688e-01,8.456524e-01,8.124684e-02
std,1.633561e+07,4.651471e-01,1.029957e+05,3.715601e+07,3.082674e+07,4.810308e+07,3.654908e+01,3.449274e+00,4.916989e-01,4.811436e-01,3.332029e-01,2.503711e-01,2.214681e-01,4.078563e-01,2.039340e-01,5.698972e-02,8.503004e-02,4.253559e-01,2.505723e-02,1.736396e-02,4.416346e-01,1.634002e-01,2.435167e-02,2.093751e-01,3.552317e-01,1.122965e-01,1.871586e-01,2.691130e-01,4.379321e-02,1.345284e-01,1.743852e-01,1.703936e-01,2.511947e-01,2.871872e-01,2.902171e-02,1.885544e-01,2.134643e-02,2.280585e-01,1.473060e-01,4.345483e-01,7.544424e-02,1.464903e-01,2.000020e-01,6.086727e-03,8.596831e-02,3.447622e-01,3.612816e-01,1.164136e-01
min,2.141000e+03,0.000000e+00,6.200000e+01,5.957590e+08,4.877100e+04,1.010649e+06,1.399000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.067120e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.075976e+07,0.000000e+00,9.680000e+02,6.233632e+08,4.149759e+07,7.426651e+07,3.499000e+01,2.960000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.499879e-01,1.000000e+00,4.934156e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.896552e-03,0.000000e+00,1.000000e+00,2.135245e-02
50%,1.651243e+07,0.000000e+00,9.680000e+02,6.539876e+08,8.053986e+07,1.371596e+08,5.099000e+01,4.700000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.499685e-01,1.000000e+00,8.913952e-02,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+0

In [36]:
main_data_v2.to_csv(file_path+'main_data_final.csv')

This is our final dataframe which we are going to train our ML model on. \
Please continue from 'model_construction_v2.ipynb'

******************************************************************************